In [1]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)


from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")

In [3]:
chat = ChatOpenAI(
    openai_api_key= OPENAI_API_KEY,
    model='gpt-3.5-turbo'
)

# Restart Here

In [4]:
messages = [
    SystemMessage(content="""You are a helpful chatbot that interact with students with queries.
                            Make sure the students their UID and email address before solving their queries and save them. You don't need to verify if they are accurate.
                            If you are given that can't be done by you such as sending mails, state that a request has been placed for this to the relevant department
                            You will be provided context to answer the questions asked of you. If the information is not enough, you can ask the student to eloberate their query.
                            Sometimes the context is previous conversations that you can use as a referance of how to reply back to the students.
                            Don't mention anything about being given context, even if it is provided in prompt."""),
]

In [5]:
res = chat(messages)
res

AIMessage(content='Hello! How can I assist you today?')

In [6]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Can you help me with any question?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Of course! I'll do my best to assist you. What question do you have?


In [7]:
import pinecone

# get API key from app.pinecone.io and environment from console
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  
    environment=os.getenv('PINECONE_ENV')  
)

C:\Users\Uday Ganti\anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
index_list = pinecone.list_indexes()

In [9]:
import time

if PINECONE_INDEX_NAME not in pinecone.list_indexes():
    pinecone.create_index(
        PINECONE_INDEX_NAME,
        dimension=1536,
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(PINECONE_INDEX_NAME).status['ready']:
        time.sleep(1)
        

index = pinecone.Index(PINECONE_INDEX_NAME)

In [10]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.2205,
 'namespaces': {'': {'vector_count': 22050}},
 'total_vector_count': 22050}

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

In [12]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

C:\Users\Uday Ganti\anaconda3\lib\site-packages\langchain\vectorstores\pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [13]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [14]:
from langchain.vectorstores import Pinecone

query = "Tell me about the practice center projects available for students? Also, tell me if I can do an online course in my final semster? Help me find an offline subject related to supply chains."

vectorstore.similarity_search(query, k=3)

[Document(page_content="'question000001': 'Hello, I have got the permit to register for MS SCX courses : Supply chain management CRN: 81618 Logistics Systems and Analytics CRN: 81619 Lean Six Sigma CRN: 81404 I think these are hybrid classes, but I wanted to know whether they are considered as offline classes as we must take at least 2 offline classes for the fall semester. And can we take these subjects in the spring semester where I will be left with only one subject. Please clarify me on this. Regards, .', 'response000002': 'Thank you for reaching out. Typically, these courses are in person. Did you send request already? Best, ', 'question000003': 'Hello, Yes, I have registered myself for Chain management. I had a query about whether hybrid class is considered as an offline class or not. Thank you,   '"),
 Document(page_content="'question000001': 'Hello, I have got the permit to register for MS SCX courses : Supply chain management CRN: 81618 Logistics Systems and Analytics CRN: 816

In [15]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    'question000001': 'Hello, I have got the permit to register for MS SCX courses : Supply chain management CRN: 81618 Logistics Systems and Analytics CRN: 81619 Lean Six Sigma CRN: 81404 I think these are hybrid classes, but I wanted to know whether they are considered as offline classes as we must take at least 2 offline classes for the fall semester. And can we take these subjects in the spring semester where I will be left with only one subject. Please clarify me on this. Regards, .', 'response000002': 'Thank you for reaching out. Typically, these courses are in person. Did you send request already? Best, ', 'question000003': 'Hello, Yes, I have registered myself for Chain management. I had a query about whether hybrid class is considered as an offline class or not. Thank you,   '
'question000001': 'Hello, I have got the permit to register for MS SCX courses : Supply chain management CRN: 81618 Logistics Systems and Analyt

In [16]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

I apologize for any confusion, but I am not able to access specific information about practice center projects or specific courses. However, I can provide you with some general information.

Practice center projects are typically hands-on projects that allow students to apply their knowledge and skills in a real-world setting. These projects provide valuable practical experience and often involve collaborating with industry partners.

Regarding your question about taking an online course in your final semester, I'm afraid I don't have that information. It would be best to consult the MS BAIS Current Students Canvas Course or reach out to the Graduate Coordinator for more details. They will be able to provide you with the most accurate and up-to-date information regarding course requirements.

As for finding an offline subject related to supply chains, I recommend checking the course catalog or speaking with your academic advisor. They will be able to guide you in selecting an appropria

In [17]:
# now create a new user prompt
prompt = HumanMessage(
    content="What if the course is a hybrid between offline and online?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

messages.append(res)

If a course is a hybrid between offline and online, it means that it includes both in-person and online components. In this case, it would be considered as an offline class since it requires physical attendance for certain activities or sessions. 

As for taking an online course in your final semester, I'm sorry to inform you that according to the information provided in the MS BAIS Current Students Canvas Course, it is not allowed to take an online course in your final semester. Offline classes are mandatory for the final semester.

Regarding finding an offline subject related to supply chains, I can assist you with that. Please give me a moment to check the available subjects.
